In [1]:
from sklearn.datasets import make_classification
import torch

In [2]:
x,y = make_classification(
    n_samples=500,
    n_features=10,
    n_informative=2,
    n_redundant=0,
    n_classes=2,
    random_state=42
)

In [49]:
x.shape

(500, 10)

In [3]:
x,y = torch.tensor(x,dtype=torch.float32), torch.tensor(y,dtype=torch.float32)  

In [51]:
y.dtype

torch.float32

In [4]:
from torch.utils.data import Dataset, DataLoader

In [5]:
class CustomDataset(Dataset):
    def __init__(self,feature,lable):
        super().__init__()
        self.feature = feature
        self.lable = lable
    def __len__(self):
        return len(self.feature)
    def __getitem__(self,index):
        return self.feature[index],self.lable[index]
    
dataset = CustomDataset(x,y)

dataloader = DataLoader(dataset,batch_size=32,shuffle=True)


In [6]:
for dat,lable in dataloader:
    print(dat[1])

tensor([ 1.5193, -0.6578,  0.8902,  0.5108,  1.5279, -0.6589, -1.0919, -0.9078,
         0.8545,  1.0306])
tensor([ 0.4754, -0.7718,  0.6360,  0.6325, -0.1551,  2.2144,  0.2275, -0.8524,
         1.6095, -0.4742])
tensor([-0.8285, -0.1759,  0.2307, -0.1033,  2.6997, -1.2491,  0.0209, -1.6009,
         1.6615, -1.6432])
tensor([-1.4056, -0.0791,  0.6962, -0.9850, -0.6486,  2.4316, -0.4808,  0.0226,
        -0.5791,  0.1987])
tensor([ 7.0149e-01,  1.6190e-03, -2.8509e-01,  5.1887e-01, -1.6480e+00,
         3.3114e+00,  1.6043e+00, -1.3650e+00,  4.1477e-01,  4.9581e-01])
tensor([ 0.6452, -0.1973,  1.1338,  0.5556, -0.1246, -0.3273, -0.1949,  0.5211,
        -0.1513,  0.0896])
tensor([ 0.7190, -1.4218, -0.2508,  0.9960, -1.1009, -1.0674, -0.3227,  1.1857,
         1.5013, -0.7568])
tensor([ 0.0723,  1.3511,  0.2431, -1.5002,  2.1662, -0.0198, -0.0626, -0.1322,
         0.8862, -0.7788])
tensor([ 0.2392, -2.5303,  1.9583,  2.0741, -0.1537, -0.3389,  1.1010, -1.0727,
        -0.2860, -0.9194

In [7]:
import torch.nn as nn
import torch.optim as optim

class SimpleNN(nn.Module):
    def __init__(self,input_layer,hidden_layer,output_layer):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_layer,hidden_layer),
            nn.ReLU(),
            nn.Linear(hidden_layer,output_layer),
            nn.Sigmoid()
        )
    def forward(self,x):
        return self.network(x)

model = SimpleNN(10,15,1)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(),lr=0.01)

for epoch in range(100):
    for x,y in dataloader:
        yhat = model(x)
        loss = criterion(yhat,y.view(-1,1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/100], Loss: {loss.item():.4f}") 

Epoch [10/100], Loss: 0.2806
Epoch [20/100], Loss: 0.1421
Epoch [30/100], Loss: 0.3765
Epoch [40/100], Loss: 0.0873
Epoch [50/100], Loss: 0.1250
Epoch [60/100], Loss: 0.2743
Epoch [70/100], Loss: 0.2984
Epoch [80/100], Loss: 0.3485
Epoch [90/100], Loss: 0.0400
Epoch [100/100], Loss: 0.1440


In [8]:
model.eval()
accuracy_list = []
with torch.no_grad():
    for x,y in dataloader:
        ypred = model(x)
        ypred = (ypred > 0.8).float()
        batch_Acc = (ypred.view(-1) == y).float().mean().item()
        accuracy_list.append(batch_Acc)
overall_Acc = sum(accuracy_list)/len(accuracy_list)
print(f"Accuracy {overall_Acc:.4f}")

Accuracy 0.9676
